In [1]:
## libraries
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
import nltk
from nltk.corpus import wordnet
import copy

In [2]:
## data loading
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [ ]:
#ftd.dtype.names

In [3]:
#changing or adding names for the dtypes
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

#fe.dtype.names
#fe[971]

193

In [4]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [5]:
all_x = mk_list(fe['pre_que'])
len(all_x)


15131

In [6]:
##check blanks
## As there are some rows where there is only one image or blank/erronous question. We check that and save those indices.
## create blank list
#good_index = ['True']*len(all_x)

bl =[]

for img,q in zip(enumerate(all_x), mk_list(qa['que'])):
    if (len(img[1]) <2 or len(q) < 2) and img[0] not in bl:
        bl.append(img[0])
print(bl)


[10, 43, 114, 115, 117, 118, 130, 131, 134, 135, 136, 137, 138, 139, 140, 234, 358, 408, 537, 607, 636, 674, 923, 928, 929, 931, 932, 949, 1078, 1079, 1196, 1250, 1251, 1258, 1264, 1274, 1364, 1433, 1493, 1494, 1545, 1600, 1753, 1907, 1996, 2045, 2077, 2095, 2103, 2105, 2106, 2167, 2168, 2169, 2201, 2308, 2312, 2413, 2455, 2535, 2536, 2542, 2657, 2681, 2706, 2772, 2773, 2774, 2778, 2779, 2781, 2782, 2783, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 2798, 2873, 2876, 2902, 2942, 2949, 2965, 3079, 3201, 3207, 3258, 3308, 3411, 3416, 3426, 3439, 3440, 3444, 3445, 3446, 3449, 3496, 3506, 3507, 3549, 3600, 3602, 3610, 3612, 3616, 3619, 3621, 3624, 3626, 3628, 3652, 3654, 3655, 3656, 3665, 3682, 3715, 3726, 3727, 3730, 3733, 3735, 3829, 3912, 3954, 3959, 4001, 4002, 4003, 4004, 4005, 4006, 4007, 4008, 4024, 4025, 4302, 4335, 4363, 4415, 4450, 4522, 4523, 4536, 4572, 4621, 4723, 4734, 4736, 4737, 4792, 4793, 4794, 4823, 4824, 4825, 4826, 4827, 4828, 4829, 4830, 4898, 4904, 4916, 4989, 5019, 504

In [7]:
len(all_x)
len(mk_list(qa['que']))
        
     

15131

In [8]:
## Now for each row in the list above, find description and add it
all_i_txt = []
original_ind = []
bl_ind = []

for rown in range(len(all_x)):
    if rown in bl:
        bl_ind.append(rown)
    else:
        #print(all_x.index)
        int_i_txt = []
        for img in all_x[rown]:
            if img in ftd['img_id']:
                d = str(ftd['description'][list(ftd['img_id']).index(img)])
                int_i_txt.append(str(d))
        original_ind.append(rown)
        all_i_txt.append(int_i_txt)
        
            

In [9]:
#print check
#print(type(all_i_txt[0]))
#print(len(all_i_txt))
#print(all_i_txt[971])
#print(all_i_txt[1473])
print(len(original_ind))
print(len(all_i_txt))
print(len(bl_ind))
print(len(bl))
print(bl == bl_ind)
#print(original_ind)
#for it in original_ind:
#    print(qa['ans'][it])

14644
14644
487
487
True


In [10]:
## For each question, find the synonyms for all the adjectives, NN, NNS and CD. Then add that list of synonyms in the 
##column called question 

new_table_fd = [] ## for frequency distribution
new_que_syn = []
for rown in qa['que'] :
    #print(rown[-2])
    #new_row = []
    sents = nltk.sent_tokenize(rown.strip())
    
    lw = [nltk.word_tokenize(s) for s in sents]
    #p =[nltk.pos_tag(w) for w in lw]
    x = [kv[0] for w in lw for kv in nltk.pos_tag(w) if kv[1] in ['JJ', 'NN', 'NNS', 'CD']]
    fd = [kv[1] for w in lw for kv in nltk.pos_tag(w)]
    #y = [wordnet.synsets(words) for words in x]
    syn = [j.name() for words in x for i in wordnet.synsets(words) for j in i.lemmas()]
    new_que_syn.append(list(set(syn)))
    new_table_fd.extend([kv[0] for w in lw for kv in nltk.pos_tag(w)])

In [11]:
## add question and answer to each data point also convert the list of synonyms in the question to a string
for it in range(len(all_i_txt)):
    all_i_txt[it].extend([' '.join(map(str, new_que_syn[it])), qa['ans'][it].strip()])
    
    

In [12]:
## check addition of above objects
for i in range(len(all_i_txt[0:2])):
    print(all_i_txt[i][-2:])

['face_fungus byssus beard whiskers', 'No']
['wear tire_out get_into looking_glass wearable methamphetamine wearing tire meth specs fag_out don spectacles have_on wear_off weary fall_apart ice glasses glaze_over Methedrine trash deoxyephedrine glassful habiliment wear_down glass_in article_of_clothing bear wear_upon methamphetamine_hydrochloride break assume spyglass glass wear_thin endure glaze drinking_glass chicken_feed hold_out clothing put_on fatigue wear_out vesture bust fag jade field_glass crank eyeglasses outwear chalk glass_over shabu', 'Yes']


In [ ]:
#for i in set(bl):
#    print(all_i_txt[i][-2:])

In [ ]:
#for rown,qs in zip(all_i_txt[0:2],new_que_syn[0:2]):
#    rown[-2] = qs
#    print(rown[-2])

In [13]:
#fd1 = nltk.FreqDist(new_table_fd[0:20])
fd1 = nltk.FreqDist(new_table_fd)
fd1.most_common()

[('?', 10645),
 ('person', 9149),
 ('the', 7885),
 ('have', 5230),
 ('Is', 3951),
 ('hair', 3730),
 ('a', 3561),
 ('Does', 3308),
 ('is', 3010),
 ('this', 2314),
 ('does', 2251),
 ('glasses', 2025),
 ('wearing', 1795),
 ('they', 1209),
 ('shirt', 1146),
 ('he', 937),
 ('wear', 870),
 ('she', 798),
 ('on', 751),
 ('their', 727),
 ('long', 620),
 ('white', 584),
 ('beard', 555),
 ('her', 517),
 ('you', 512),
 ('black', 510),
 ('facial', 470),
 ('short', 459),
 ('do', 452),
 ('blonde', 451),
 ('Do', 449),
 ('male', 421),
 ('are', 415),
 ('asian', 396),
 ('life', 383),
 ('Are', 367),
 ('it', 349),
 ("'s", 346),
 ('in', 317),
 ('undershirt', 314),
 ('female', 311),
 ('man', 295),
 ('girl', 289),
 ('tall', 285),
 ('dark', 278),
 ('boy', 267),
 ('his', 253),
 ('of', 220),
 ('woman', 216),
 ('smiling', 215),
 ('back', 199),
 ('up', 191),
 ('say', 181),
 ('down', 174),
 ('head', 173),
 ('with', 172),
 ('ponytail', 165),
 ('brown', 161),
 ('face', 159),
 ('an', 158),
 ('light', 156),
 ('bangs', 

In [15]:
#print(all_i_txt[0])
##basic model
tf = TfidfVectorizer(ngram_range = (2,3), sublinear_tf = True)

In [16]:
all_sim_tf = []
for l in range(len(all_i_txt)):
    H = tf.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim_tf.append(sim)

In [17]:
#similarities[-2] meaning taking only the array for question
all_sim_q_tf = []
for i in range(len(all_sim_tf)):
    all_sim_q_tf.append(all_sim_tf[i][-2][:-2])
print(len(all_sim_q_tf))
#print(all_sim_q[971])
#print(len(all_sim))

14644


In [18]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind_tf = []
for si in all_sim_q_tf:
    y_top = si.argsort()[::-1]
    sorted_ind_tf.append(y_top)
#sorted_ind[971]

In [19]:
sorted_sim_all_tf = []
for q,w in zip(sorted_ind_tf, all_sim_q_tf):
    sorted_sim = []
    for e in q:
        sorted_sim.append(w[e])
    sorted_sim_all_tf.append(sorted_sim)
print(len(sorted_sim_all_tf))

14644


In [20]:
##find the difference between two similarity score in each array.
sim_dif_all_tf = []

for d in range(len(sorted_sim_all_tf)):
    sim_dif = [sorted_sim_all_tf[d][f-1] - sorted_sim_all_tf[d][f] for f in range(1, len(sorted_sim_all_tf[d]))]
    sim_dif_all_tf.append(sim_dif)
        
len(sim_dif_all_tf)


14644

In [21]:

max_dif_tf = []
max_dif_ind_tf =[]
for g in range(len(sim_dif_all_tf)):
    
    maxi= max(sim_dif_all_tf[g])
    max_dif_tf.append(maxi)
    max_dif_ind_tf.append(sim_dif_all_tf[g].index(maxi))
print(len(max_dif_tf))
print(len(max_dif_ind_tf))

14644
14644


In [22]:
sorted_ind_lim_tf = [indi[0:max_indi+1] for indi,max_indi in zip(sorted_ind_tf, max_dif_ind_tf)]


In [23]:
##clean_all_x
all_x_clean = [all_x[i] for i in original_ind]

In [24]:
top_img = zip(all_x_clean, sorted_ind_lim_tf)

In [25]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred

In [26]:
pred_img = mk_imgid(top_img)

In [27]:
len(pred_img)
pred_img[0]

['IMG_9048']

In [28]:
all_cheat1 = mk_list(fe['curr_elim'])
all_cheat = [all_cheat1[i] for i in original_ind]

In [29]:
len(all_cheat)
all_cheat[0]

['IMG_9634', 'IMG_7942']

In [30]:
pre_all= [len(set(a).intersection(set(p)))/len(set(p)) if len(p) > 0 else 0 for a,p in zip(all_cheat, pred_img)]
rec_all = [len(set(a).intersection(set(p)))/len(set(a)) if len(a) > 0 else 0 for a,p in zip(all_cheat, pred_img)]
print(len(pre_all))
print(len(rec_all))


14644
14644


In [31]:
fs_all= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all, rec_all)]
len(fs_all)

14644

In [32]:
pre_mean = np.mean(np.array(pre_all))
rec_mean = np.mean(np.array(rec_all))
fs_mean = np.mean(np.array(fs_all))
pre_mean

0.4256663707548029

In [ ]:
#with open('Results with description.txt', 'w') as f:
#    f.write("No., Model, Parameter, Decision Rules, Precision, Recall, F0.5"+ '\n')

In [33]:
op3 = ["17", "TFIDF, NLTK, POS", "ngram_range = 2,3, sublineartf true", "Cosinesimilarity and ranking difference",str(pre_mean), str(rec_mean), str(fs_mean)]

In [34]:
with open('Results with description.txt', 'a') as f:
    f.write(', '.join(op3)+ '\n')

In [50]:
cv = CountVectorizer()

In [51]:
## all similarities
all_sim_cv = []
for l in range(len(all_i_txt)):
    H = cv.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim_cv.append(sim)

In [52]:
#similarities[-2] meaning taking only the array for question
all_sim_q_cv = []
for i in range(len(all_sim_cv)):
    all_sim_q_cv.append(all_sim_cv[i][-2][:-2])
print(len(all_sim_q_cv))
#print(all_sim_q[971])
#print(len(all_sim))

14644


In [53]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind_cv = []
for si in all_sim_q_cv:
    y_top = si.argsort()[::-1]
    sorted_ind_cv.append(y_top)
#sorted_ind[971]

In [54]:
sorted_sim_all_cv = []
for q,w in zip(sorted_ind_cv, all_sim_q_cv):
    sorted_sim = []
    for e in q:
        sorted_sim.append(w[e])
    sorted_sim_all_cv.append(sorted_sim)
print(len(sorted_sim_all_cv))

14644


In [55]:
##find the difference between two similarity score in each array.
sim_dif_all_cv = []

for d in range(len(sorted_sim_all_cv)):
    sim_dif = [sorted_sim_all_cv[d][f-1] - sorted_sim_all_cv[d][f] for f in range(1, len(sorted_sim_all_cv[d]))]
    sim_dif_all_cv.append(sim_dif)
        
len(sim_dif_all_cv)


14644

In [57]:

max_dif_cv = []
max_dif_ind_cv =[]
for g in range(len(sim_dif_all_cv)):
    
    maxi= max(sim_dif_all_cv[g])
    max_dif_cv.append(maxi)
    max_dif_ind_cv.append(sim_dif_all_cv[g].index(maxi))
print(len(max_dif_cv))
print(len(max_dif_ind_cv))

14644
14644


In [58]:
sorted_ind_lim_cv = [indi[0:max_indi+1] for indi,max_indi in zip(sorted_ind_cv, max_dif_ind_cv)]


In [59]:
top_img_cv = zip(all_x_clean, sorted_ind_lim_cv)

In [60]:
pred_img_cv = mk_imgid(top_img_cv)

In [67]:
pre_all_cv= [len(set(a).intersection(set(p)))/len(set(p)) if len(p) > 0 else 0 for a,p in zip(all_cheat, pred_img_cv)]
rec_all_cv = [len(set(a).intersection(set(p)))/len(set(a)) if len(a) > 0 else 0 for a,p in zip(all_cheat, pred_img_cv)]
print(len(pre_all_cv))
print(len(rec_all_cv))


14644
14644


In [68]:
fs_all_cv= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all_cv, rec_all_cv)]
len(fs_all_cv)

14644

In [69]:
pre_mean_cv = np.mean(np.array(pre_all_cv))
rec_mean_cv = np.mean(np.array(rec_all_cv))
fs_mean_cv = np.mean(np.array(fs_all_cv))
pre_mean_cv

0.031238461491466137

In [70]:
op3 = ["18", "cv, NLTK, POS", "default", "Cosinesimilarity and ranking difference",str(pre_mean_cv), str(rec_mean_cv), str(fs_mean_cv)]

In [71]:
with open('Results with description.txt', 'a') as f:
    f.write(', '.join(op3)+ '\n')